In [ ]:
import os
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import (
    roc_auc_score, precision_recall_curve, roc_curve, auc,
    confusion_matrix, cohen_kappa_score, matthews_corrcoef, f1_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
import psutil
import warnings
from rdkit import RDLogger
import joblib

# 环境设置
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings("ignore")
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 300
sns.set_style('whitegrid')

# ---------------------- 内存安全装饰器 ----------------------
def memory_safe(func):
    def wrapper(*args, **kwargs):
        mem = psutil.virtual_memory()
        if mem.percent > 80:
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            print(f"⚠️ Memory warning: Usage {mem.percent}%, performed garbage collection")
        return func(*args, **kwargs)
    return wrapper

# ---------------------- SMILES特征提取 ----------------------
class SMILESFeatureExtractor:
    def __init__(self, fp_size=1024, desc_list=None):
        self.fp_size = fp_size
        self.desc_list = desc_list or [
            'MolWt', 'NumHAcceptors', 'NumHDonors', 
            'MolLogP', 'TPSA', 'NumRotatableBonds'
        ]
    
    @memory_safe
    def smiles_to_features(self, smiles):
        """Convert SMILES to numerical features"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if not mol:
                return np.nan * np.ones(len(self.desc_list) + self.fp_size)
            
            # 计算描述符
            desc_values = [getattr(Descriptors, desc)(mol) for desc in self.desc_list]
            
            # 计算指纹
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=self.fp_size)
            fp_values = np.array(fp, dtype=np.float32)
            
            return np.concatenate([desc_values, fp_values])
        except:
            return np.nan * np.ones(len(self.desc_list) + self.fp_size)

# ---------------------- 数据准备 ----------------------
def prepare_features(X_smiles):
    """Convert SMILES pairs to numerical features"""
    fe = SMILESFeatureExtractor()
    features = []
    
    for drug1, drug2 in tqdm(X_smiles, desc="Extracting features"):
        feat1 = fe.smiles_to_features(drug1)
        feat2 = fe.smiles_to_features(drug2)
        features.append(np.concatenate([feat1, feat2]))
    
    X_num = np.stack(features)
    return np.nan_to_num(X_num)

# ---------------------- 深度学习组件 ----------------------
class DrugInteractionDataset(Dataset):
    def __init__(self, drug1_smiles, drug2_smiles, labels=None, tokenizer=None, max_length=128):
        self.drug1_smiles = drug1_smiles
        self.drug2_smiles = drug2_smiles
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.drug1_smiles)
    
    def __getitem__(self, idx):
        encoding1 = self.tokenizer(
            str(self.drug1_smiles[idx]), 
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        encoding2 = self.tokenizer(
            str(self.drug2_smiles[idx]),
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        item = {
            'drug1_input_ids': encoding1['input_ids'].flatten(),
            'drug1_attention_mask': encoding1['attention_mask'].flatten(),
            'drug2_input_ids': encoding2['input_ids'].flatten(),
            'drug2_attention_mask': encoding2['attention_mask'].flatten(),
        }
        
        if self.labels is not None:
            item['label'] = torch.tensor(self.labels[idx], dtype=torch.long)
            
        return item

class CoAttentionModel(nn.Module):
    def __init__(self, bert_model_name="DeepChem/ChemBERTa-77M-MLM", hidden_size=384):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.co_attention = nn.MultiheadAttention(hidden_size, num_heads=8)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size*4, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, 2)
        )
    
    def forward(self, drug1_input_ids, drug1_attention_mask, drug2_input_ids, drug2_attention_mask):
        drug1 = self.bert(drug1_input_ids, attention_mask=drug1_attention_mask).last_hidden_state[:, 0, :]
        drug2 = self.bert(drug2_input_ids, attention_mask=drug2_attention_mask).last_hidden_state[:, 0, :]
        
        # 协同注意力机制
        attn1, _ = self.co_attention(drug1.unsqueeze(1), drug2.unsqueeze(1), drug2.unsqueeze(1))
        attn2, _ = self.co_attention(drug2.unsqueeze(1), drug1.unsqueeze(1), drug1.unsqueeze(1))
        
        combined = torch.cat([drug1, drug2, attn1.squeeze(1), attn2.squeeze(1)], dim=1)
        return self.classifier(combined)

# ---------------------- 评估相关函数 ----------------------
def find_optimal_cutoff(tpr, fpr, thresholds):
    """使用Youden's J统计量寻找最佳阈值"""
    youden = tpr - fpr
    return thresholds[np.argmax(youden)]

def best_confusion_matrix(y_true, y_prob, y_train=None, train_prob=None):
    """
    计算最佳阈值和混淆矩阵
    优先级: 测试集ROC > 训练集ROC > 默认0.5
    """
    try:
        # 优先使用测试集数据
        fpr, tpr, thresholds = roc_curve(y_true, y_prob, pos_label=1)
        cutoff = find_optimal_cutoff(tpr, fpr, thresholds)
        y_pred = (y_prob >= cutoff).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        return cutoff, tn, fp, fn, tp
    except ValueError:
        # 测试集全0/全1时，尝试使用训练集阈值
        if y_train is not None and train_prob is not None:
            try:
                fpr, tpr, thresholds = roc_curve(y_train, train_prob, pos_label=1)
                cutoff = find_optimal_cutoff(tpr, fpr, thresholds)
                print(f"⚠️ 使用训练集确定的最佳阈值: {cutoff:.4f}")
                y_pred = (y_prob >= cutoff).astype(int)
                if np.all(y_true == 1):  # 处理全1标签的情况
                    tp = np.sum(y_pred == 1)
                    fn = len(y_true) - tp
                    return cutoff, 0, 0, fn, tp
                elif np.all(y_true == 0):  # 处理全0标签的情况
                    tn = np.sum(y_pred == 0)
                    fp = len(y_true) - tn
                    return cutoff, tn, fp, 0, 0
            except:
                pass
        
        # 最终回退到默认阈值
        print("⚠️ 使用默认阈值0.5")
        cutoff = 0.5
        y_pred = (y_prob >= cutoff).astype(int)
        if np.all(y_true == 1):  # 处理全1标签的情况
            tp = np.sum(y_pred == 1)
            fn = len(y_true) - tp
            return cutoff, 0, 0, fn, tp
        elif np.all(y_true == 0):  # 处理全0标签的情况
            tn = np.sum(y_pred == 0)
            fp = len(y_true) - tn
            return cutoff, tn, fp, 0, 0
        else:
            return cutoff, 0, 0, 0, 0  # 其他未知情况

@memory_safe
@memory_safe
def evaluate_model(model, data_loader, criterion, device, y_train=None, train_prob=None):
    """综合评估模型性能"""
    model.eval()
    total_loss = 0
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for batch in data_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            outputs = model(**inputs)
            loss = criterion(outputs, batch['label'].to(device))
            total_loss += loss.item()
            
            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            all_labels.extend(batch['label'].cpu().numpy())
            all_probs.extend(probs)
    
    # 转换为numpy数组
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    # 处理极端情况
    unique_labels = np.unique(all_labels)
    if len(unique_labels) == 1:
        print(f"⚠️ 测试集标签全为{unique_labels[0]}，启用后备指标计算")
        # 对于单一类别，设置默认指标值
        if unique_labels[0] == 1:  # 全1标签
            metrics = {
                "AUC": 1.0,
                "Cutoff": 0.5,
                "Sensitivity": 1.0,
                "Specificity": 0.0,
                "Precision": 1.0,
                "F1": 1.0,
                "MCC": 0.0,
                "Kappa": 0.0,
            }
        else:  # 全0标签
            metrics = {
                "AUC": 1.0,
                "Cutoff": 0.5,
                "Sensitivity": 0.0,
                "Specificity": 1.0,
                "Precision": 0.0,
                "F1": 0.0,
                "MCC": 0.0,
                "Kappa": 0.0,
            }
        return total_loss / len(data_loader), metrics
    
    # 正常情况下的指标计算
    cutoff, tn, fp, fn, tp = best_confusion_matrix(all_labels, all_probs, y_train, train_prob)
    
    metrics = {
        "AUC": roc_auc_score(all_labels, all_probs),
        "Cutoff": cutoff,
        "Sensitivity": tp / (tp + fn) if (tp + fn) > 0 else 0.0,
        "Specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
        "Precision": tp / (tp + fp) if (tp + fp) > 0 else 0.0,
        "F1": f1_score(all_labels, (all_probs >= cutoff).astype(int)),
        "MCC": matthews_corrcoef(all_labels, (all_probs >= cutoff).astype(int)),
        "Kappa": cohen_kappa_score(all_labels, (all_probs >= cutoff).astype(int)),
    }
    
    return total_loss / len(data_loader), metrics


# ---------------------- 训练函数 ----------------------
@memory_safe
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10):
    """训练深度学习模型"""
    best_val_loss = float('inf')
    best_model = None
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_train_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            outputs = model(**inputs)
            loss = criterion(outputs, batch['label'].to(device))
            
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item()
        
        # 计算验证损失
        val_loss, _ = evaluate_model(model, val_loader, criterion, device)
        
        # 记录损失
        train_losses.append(epoch_train_loss / len(train_loader))
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1} - Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")
        
        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
    
    # 加载最佳模型
    model.load_state_dict(best_model)
    return model, (train_losses, val_losses)

# ---------------------- 预测函数 ----------------------
@memory_safe
def predict(model, data_loader, device):
    """使用模型进行预测"""
    model.eval()
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            outputs = model(**inputs)
            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()

            all_probs.extend(probs)
            
            if 'label' in batch:
                all_labels.extend(batch['label'].cpu().numpy())
    
    return np.array(all_probs), np.array(all_labels) if len(all_labels) > 0 else None

# ---------------------- 保存预测结果函数 ----------------------
def save_prediction_scores(model_name, smiles_pairs, probs, labels, cutoff):
    """保存模型预测结果"""
    df = pd.DataFrame({
        'Drug1_SMILES': smiles_pairs[:, 0],
        'Drug2_SMILES': smiles_pairs[:, 1],
        'Predicted_Probability': probs,
        'Predicted_Label': (probs >= cutoff).astype(int)
    })
    
    if labels is not None:
        df['True_Label'] = labels
    
    df.to_csv(f"{model_name.replace(' ', '_')}_predictions.csv", index=False)
    print(f"✅ {model_name} 预测结果已保存")

# ---------------------- 主执行流程 ----------------------
def main():
    # 数据加载
    train_df = pd.read_csv("/kaggle/working/3/3-version2/train.txt", sep='\t', header=None)
    X_train_smiles = train_df.iloc[:, :2].values
    y_train = train_df.iloc[:, -1].values
    
    test_df = pd.read_csv("/kaggle/working/3/3-version2/test.txt", sep='\t', header=None)
    X_test_smiles = test_df.iloc[:, :2].values
    y_test = test_df.iloc[:, -1].values if test_df.shape[1] > 2 else None



    
    # 检查数据平衡性
    print("训练集类别分布:", np.unique(y_train, return_counts=True))
    if y_test is not None:
        print("测试集类别分布:", np.unique(y_test, return_counts=True))




    ##########
    
    # 特征工程
    print("\nPreparing features...")
    X_train_num = prepare_features(X_train_smiles)
    X_test_num = prepare_features(X_test_smiles)
    
    scaler = StandardScaler()
    X_train_num = scaler.fit_transform(X_train_num)
    X_test_num = scaler.transform(X_test_num)

    # 模型定义
    classifiers = {
        "ChemCoBERT": None,
        "Decision Tree": DecisionTreeClassifier(max_depth=5, random_state=42),
        "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
        "GBDT": GradientBoostingClassifier(n_estimators=100, random_state=42),
        "K-NN": KNeighborsClassifier(n_neighbors=5),
        "Naive Bayes": GaussianNB(var_smoothing=1e-2)
    }
    
    # 训练和评估

    # 训练和评估
    all_metrics = {}
    pr_curves = {}
    roc_curves = {}
    
    for name, clf in classifiers.items():
        print(f"\n{'='*50}\nTraining {name}...\n{'='*50}")
        
        if name == "ChemCoBERT":
            # 深度学习模型流程
            tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
            train_dataset = DrugInteractionDataset(X_train_smiles[:, 0], X_train_smiles[:, 1], y_train, tokenizer)
            test_dataset = DrugInteractionDataset(X_test_smiles[:, 0], X_test_smiles[:, 1], y_test, tokenizer)
            
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = CoAttentionModel().to(device)
            optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
            criterion = nn.CrossEntropyLoss()
            
            # 训练
            model, _ = train_model(
                model, 
                DataLoader(train_dataset, batch_size=8, shuffle=True),
                DataLoader(test_dataset, batch_size=8),
                optimizer, criterion, device, epochs=10
            )
            
            # 评估
            if y_test is not None:
                # 获取训练集预测概率（用于后备阈值）
                train_probs, _ = predict(model, DataLoader(train_dataset, batch_size=128), device)
                
                # 评估测试集
                _, test_metrics = evaluate_model(
                    model, 
                    DataLoader(test_dataset, batch_size=8),
                    criterion, 
                    device,
                    y_train,
                    train_probs
                )
                all_metrics[name] = test_metrics
                
                # 保存预测结果
                test_probs, _ = predict(model, DataLoader(test_dataset, batch_size=128), device)
                save_prediction_scores(name, X_test_smiles, test_probs, y_test, test_metrics['Cutoff'])
            
            # 保存模型
            torch.save(model.state_dict(), f"{name.replace(' ', '_')}_model.pth")

        else:
            # 传统机器学习流程
            clf.fit(X_train_num, y_train)
            joblib.dump(clf, f"{name.replace(' ', '_')}_model.joblib")
            
            if y_test is not None:
                # 获取训练集预测概率
                if hasattr(clf, "predict_proba"):
                    train_probs = clf.predict_proba(X_train_num)[:, 1]
                else:
                    train_probs = clf.decision_function(X_train_num)
                    train_probs = (train_probs - train_probs.min()) / (train_probs.max() - train_probs.min())
                
                # 测试集预测
                if hasattr(clf, "predict_proba"):
                    test_probs = clf.predict_proba(X_test_num)[:, 1]
                else:
                    test_probs = clf.decision_function(X_test_num)
                    test_probs = (test_probs - test_probs.min()) / (test_probs.max() - test_probs.min())
                
                # 计算指标
                cutoff, tn, fp, fn, tp = best_confusion_matrix(
                    y_test, test_probs, y_train, train_probs
                )
                
                all_metrics[name] = {
                    "AUC": roc_auc_score(y_test, test_probs) if len(np.unique(y_test)) > 1 else 0.5,
                    "Cutoff": cutoff,
                    "Sensitivity": tp / (tp + fn) if (tp + fn) > 0 else 1.0 * (np.mean(y_test) == 1),
                    "Specificity": tn / (tn + fp) if (tn + fp) > 0 else 1.0 * (np.mean(y_test) == 0),
                    "Precision": tp / (tp + fp) if (tp + fp) > 0 else 0.0,
                    "F1": f1_score(y_test, (test_probs >= cutoff).astype(int)),
                    "MCC": matthews_corrcoef(y_test, (test_probs >= cutoff).astype(int)),
                    "Kappa": cohen_kappa_score(y_test, (test_probs >= cutoff).astype(int)),
                }
                
                # 保存预测结果
                save_prediction_scores(name, X_test_smiles, test_probs, y_test, cutoff)
    
    # 输出所有模型的评估结果
    print("\n\n=== 模型性能比较 ===")
    metrics_df = pd.DataFrame(all_metrics).T
    print(metrics_df)
    
    # 保存评估结果
    metrics_df.to_csv("model_performance_comparison.csv")
    print("\n✅ 所有模型评估结果已保存到 model_performance_comparison.csv")

if __name__ == "__main__":
    main()
